This Notebook sets up an Ansbile Tower for personal use via Vagrant Box and uploads a lisence as well as changes the admin password as you please.

you first need to define some settings
* vagrant_dir
the vagrant dir is the directory in which you will have your vagrant file and from which you start your vagrant box
* license_file
the full path to the lisence file which will be uploaded
* new_password
the new admin password which will be set for Tower

In [ ]:
vagrant_dir=/Users/mschreie/projects/vagrant_tower
license_file=/Users/mschreie/Downloads/license_4ab5449a6fd2475eb7000d300fb056ba.txt
new_password=redhat

In [ ]:
mkdir $vagrant_dir

In [ ]:
cd $vagrant_dir

In [ ]:
vagrant init ansible/tower

In [ ]:
vagrant up

You need to get a lisence from:
- for RH internal people: https://store.ansible.com/redhat/tower_license/
- for anyone else: https://www.ansible.com/tower-trial
and let the variable "license_file point to the file downloaded. I also find username and password from within the vagrant-tower box and store it in the respective variables.

In [ ]:
user=$(vagrant ssh -c  "bash /etc/profile.d/ansible-tower.sh" 2>/dev/null | awk '/Username:/ {print $2 }' | tr -d '\r')
password=$(vagrant ssh -c  "bash /etc/profile.d/ansible-tower.sh" 2>/dev/null | awk '/Password:/ {print $2 }' | tr -d '\r')

just for the sake of knowing what's going on it is good to see all variables are set up correctly.
HINT: Maybe don't save the notebook (or upload into git) without clearing the results of the commands, otherwise you would run into a password breach. 
Maybe just skip this box.

In [ ]:
pwd
echo $license_file
echo $user
echo $password

Now upload the lisence file to Ansible Tower

In [ ]:
curl --user $user:$password --insecure -H "Content-Type: application/json" -X POST -d "$(cat $license_file | sed -e '2i\
\    "eula_accepted": "true",')" https://10.42.0.42/api/v2/config/

You need to log off, otherwise you might run into issues, that logging in won't work as expected.

In [ ]:
curl --insecure -H "Content-Type: application/json" -X GET https://10.42.0.42/api/logout/?next=/api/login/

To change the admin password to $new_password i do the following

In [ ]:
vagrant ssh -c "echo \"from django.contrib.auth.models import User; passwd='$new_password'; usr=User.objects.get(username='admin'); usr.set_password(passwd); usr.save();\" | sudo -i awx-manage shell"
